In [ ]:
# ============================================================
# GOOGLE COLAB - PIPELINE MULTI-SEGMENTO COM RELATÓRIO FINAL
# ============================================================

import os
import json
import zipfile
import unicodedata
import pandas as pd
from google.colab import files

# ============================================================
# CONFIGURAÇÕES GERAIS
# ============================================================
BASE_TRABALHO = "/content/dados_processados"
BASE_SAIDA = "/content/saida_processamento"

os.makedirs(BASE_TRABALHO, exist_ok=True)
os.makedirs(BASE_SAIDA, exist_ok=True)

# ============================================================
# FUNÇÕES UTILITÁRIAS
# ============================================================
def normalizar_nome(valor):
    if pd.isna(valor):
        return None
    valor = str(valor).strip().upper()
    valor = unicodedata.normalize("NFKD", valor).encode("ascii", "ignore").decode("utf-8")
    valor = valor.replace(":", "-").replace("/", "-").replace("\\", "-")
    valor = valor.replace(".", "").replace(",", "")
    return " ".join(valor.split())


def carregar_json(path):
    with open(path, "r", encoding="utf-8") as f:
        return json.load(f)


def carregar_xlsx(path):
    df = pd.read_excel(path, engine="openpyxl")
    df.columns = (
        df.columns
        .str.strip()
        .str.lower()
        .str.replace(" ", "_")
        .str.replace("á", "a")
        .str.replace("ã", "a")
        .str.replace("ç", "c")
    )
    return df

# ============================================================
# FUNÇÕES DE VALIDAÇÃO (COM RELATÓRIO)
# ============================================================
def validar_professores(df, professores_json, relatorio):
    relatorio.append("\n=== Validação de Professores ===")

    excel = set(df["prof_nome"].dropna())
    json_set = set(normalizar_nome(p["nome_completo"]) for p in professores_json)

    faltantes = excel - json_set

    if faltantes:
        relatorio.append(f"⚠️ Professores ausentes no JSON: {len(faltantes)}")
        for n in sorted(faltantes):
            relatorio.append(f"- {n}")
    else:
        relatorio.append("✅ Nenhum professor ausente")

    return faltantes


def validar_competencias(df, competencias_json, relatorio):
    relatorio.append("\n=== Validação de Competências ===")

    excel = set(df["comp_nome"].dropna())
    json_set = set(normalizar_nome(c["nome"]) for c in competencias_json)

    faltantes = excel - json_set

    if faltantes:
        relatorio.append(f"⚠️ Competências ausentes no JSON: {len(faltantes)}")
        for n in sorted(faltantes):
            relatorio.append(f"- {n}")
    else:
        relatorio.append("✅ Nenhuma competência ausente")

    return faltantes


def validar_subareas(df_melt, subareas_json, relatorio):
    relatorio.append("\n=== Validação de Subáreas ===")

    excel = set(df_melt["subarea_nome"].dropna())
    json_set = set(normalizar_nome(s["nome"]) for s in subareas_json)

    faltantes = excel - json_set

    if faltantes:
        relatorio.append(f"⚠️ Subáreas ausentes no JSON: {len(faltantes)}")
        for n in sorted(faltantes):
            relatorio.append(f"- {n}")
    else:
        relatorio.append("✅ Nenhuma subárea ausente")

    return faltantes

# ============================================================
# UPLOAD DOS ZIPS
# ============================================================
print("📤 Faça upload de um ou mais arquivos .zip (CIDADES, ENERGIA, TICs, etc.)")
uploaded = files.upload()

zip_files = [f for f in uploaded if f.lower().endswith(".zip")]

if not zip_files:
    raise RuntimeError("❌ Nenhum arquivo .zip enviado.")

print(f"✅ {len(zip_files)} ZIP(s) recebidos:", zip_files)

# ============================================================
# PIPELINE POR SEGMENTO
# ============================================================
for zip_name in zip_files:

    segmento = os.path.splitext(zip_name)[0].lower()
    print(f"\n🚀 Processando segmento: {segmento.upper()}")

    DIR_TRAB = f"{BASE_TRABALHO}/{segmento}"
    DIR_OUT = f"{BASE_SAIDA}/{segmento}"

    os.makedirs(DIR_TRAB, exist_ok=True)
    os.makedirs(DIR_OUT, exist_ok=True)

    relatorio_final = []
    relatorio_final.append(f"SEGMENTO: {segmento.upper()}")
    relatorio_final.append("Início do processamento")

    # --------------------------------------------------------
    # EXTRAÇÃO
    # --------------------------------------------------------
    with zipfile.ZipFile(f"/content/{zip_name}", "r") as z:
        z.extractall(DIR_TRAB)

    subdirs = [
        os.path.join(DIR_TRAB, d)
        for d in os.listdir(DIR_TRAB)
        if os.path.isdir(os.path.join(DIR_TRAB, d))
    ]

    DIR_DADOS = subdirs[0] if subdirs else DIR_TRAB
    relatorio_final.append(f"Diretório de dados: {DIR_DADOS}")

    # --------------------------------------------------------
    # IDENTIFICAÇÃO DE ARQUIVOS
    # --------------------------------------------------------
    jsons = [f for f in os.listdir(DIR_DADOS) if f.endswith(".json")]
    xlsx = [f for f in os.listdir(DIR_DADOS) if f.endswith(".xlsx")]

    paths = {"prof": None, "comp": None, "sub": None, "base": None}

    for f in jsons:
        if "prof" in f.lower():
            paths["prof"] = os.path.join(DIR_DADOS, f)
        elif "comp" in f.lower():
            paths["comp"] = os.path.join(DIR_DADOS, f)
        elif "subarea" in f.lower() or "grupo" in f.lower():
            paths["sub"] = os.path.join(DIR_DADOS, f)

    for f in xlsx:
        if "base" in f.lower() or "prof_comp" in f.lower():
            paths["base"] = os.path.join(DIR_DADOS, f)

    if not all(paths.values()):
        relatorio_final.append("❌ Arquivos obrigatórios não encontrados")
        relatorio_texto = "\n".join(relatorio_final)
        print(relatorio_texto)
        with open(f"{DIR_OUT}/relatorio_processamento.txt", "w", encoding="utf-8") as f:
            f.write(relatorio_texto)
        continue

    # --------------------------------------------------------
    # CARGA
    # --------------------------------------------------------
    professores = carregar_json(paths["prof"])
    competencias = carregar_json(paths["comp"])
    subareas = carregar_json(paths["sub"])
    df_base = carregar_xlsx(paths["base"])

    relatorio_final.append(f"Linhas na base original: {len(df_base)}")
    relatorio_final.append(f"Colunas detectadas: {list(df_base.columns)}")

    # --------------------------------------------------------
    # TRANSFORMAÇÃO
    # --------------------------------------------------------
    df = df_base.copy()
    df["prof_nome"] = df["nome_completo"].apply(normalizar_nome)
    df["comp_nome"] = df["comp"].apply(normalizar_nome)

    sub_cols = [c for c in df.columns if c.startswith("subarea")]

    df_melt = df.melt(
        id_vars=["prof_nome", "comp_nome"],
        value_vars=sub_cols,
        value_name="subarea_nome"
    ).dropna(subset=["subarea_nome"])

    df_melt["subarea_nome"] = df_melt["subarea_nome"].apply(normalizar_nome)

    # --------------------------------------------------------
    # VALIDAÇÕES
    # --------------------------------------------------------
    falt_prof = validar_professores(df, professores, relatorio_final)
    falt_comp = validar_competencias(df, competencias, relatorio_final)
    falt_sub  = validar_subareas(df_melt, subareas, relatorio_final)

    if falt_prof or falt_comp or falt_sub:
        relatorio_final.append("\n⛔ Pipeline interrompido por inconsistências.")
    else:
        relatorio_final.append("\n✅ Todas as validações passaram com sucesso.")

        # ----------------------------------------------------
        # MAPEAMENTO
        # ----------------------------------------------------
        map_prof = {normalizar_nome(p["nome_completo"]): p["id"] for p in professores}
        map_comp = {normalizar_nome(c["nome"]): c["id"] for c in competencias}
        map_sub  = {normalizar_nome(s["nome"]): s["id"] for s in subareas}

        df_melt["professor_id"] = df_melt["prof_nome"].map(map_prof)
        df_melt["comp_id"] = df_melt["comp_nome"].map(map_comp)
        df_melt["subarea_id"] = df_melt["subarea_nome"].map(map_sub)

        df_final = (
            df_melt
            .dropna(subset=["professor_id", "comp_id", "subarea_id"])
            [["professor_id", "comp_id", "subarea_id"]]
            .astype(int)
            .drop_duplicates()
        )

        relatorio_final.append("\n=== Geração de Combinações ===")
        relatorio_final.append(f"Total após unpivot: {len(df_melt)}")
        relatorio_final.append(f"Combinações válidas geradas: {len(df_final)}")
        relatorio_final.append(
            f"Combinações descartadas: {len(df_melt) - len(df_final)}"
        )

        # ----------------------------------------------------
        # SAÍDAS
        # ----------------------------------------------------
        df_final.to_excel(f"{DIR_OUT}/prof_comp_subarea.xlsx", index=False)
        df_final.to_csv(f"{DIR_OUT}/prof_comp_subarea.csv", index=False)

        with open(f"{DIR_OUT}/insert_prof_comp_subarea.sql", "w") as f:
            for _, r in df_final.iterrows():
                f.write(
                    f"INSERT INTO public.prof_comp_subarea "
                    f"(professor_id, comp_id, subarea_id) "
                    f"VALUES ({r.professor_id}, {r.comp_id}, {r.subarea_id});\n"
                )

    # --------------------------------------------------------
    # RELATÓRIO FINAL
    # --------------------------------------------------------
    relatorio_texto = "\n".join(relatorio_final)
    print("\n" + "=" * 60)
    print(relatorio_texto)
    print("=" * 60)

    with open(f"{DIR_OUT}/relatorio_processamento.txt", "w", encoding="utf-8") as f:
        f.write(relatorio_texto)

print("\n🏁 PIPELINE MULTI-SEGMENTO FINALIZADO")


📤 Faça upload de um ou mais arquivos .zip (CIDADES, ENERGIA, TICs, etc.)


Saving CIDADES.zip to CIDADES (1).zip
✅ 1 ZIP(s) recebidos: ['CIDADES (1).zip']

🚀 Processando segmento: CIDADES (1)

SEGMENTO: CIDADES (1)
Início do processamento
Diretório de dados: /content/dados_processados/cidades (1)/CIDADES
Linhas na base original: 26
Colunas detectadas: ['nome_completo', 'comp', 'subarea1', 'subarea2', 'subarea3', 'subarea4']

=== Validação de Professores ===
✅ Nenhum professor ausente

=== Validação de Competências ===
✅ Nenhuma competência ausente

=== Validação de Subáreas ===
✅ Nenhuma subárea ausente

✅ Todas as validações passaram com sucesso.

=== Geração de Combinações ===
Total após unpivot: 41
Combinações válidas geradas: 41
Combinações descartadas: 0

🏁 PIPELINE MULTI-SEGMENTO FINALIZADO
